In [1]:
#!pip install python-dotenv
##!pip install requests
#!pip install pandas
#!pip install loguru
#!pip install datetime
#!pip install numpy
#!pip install beautifulsoup4
#!pip install sqlalchemy

In [1]:
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import psycopg2
from psycopg2.extras import Json, execute_values
import requests
import logging
from datetime import datetime

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#### EXTRACTION

In [ ]:
import os
import time
import requests
import pandas as pd
from dotenv import load_dotenv

# Load API key from .env (ensure your .env contains API_KEY=your_key)
load_dotenv()
API_KEY = os.getenv("API_KEY")
if not API_KEY:
    raise RuntimeError("API_KEY not found in environment. Put API_KEY=... in your .env file")

# Base URLs
BASE_PROPERTY_URL = "https://api.rentcast.io/v1/properties"
SALE_LISTING_URL = "https://api.rentcast.io/v1/listings/sale"

# Common headers
headers = {
    "accept": "application/json",
    "x-api-key": API_KEY
}

# Storage
property_records = []
sale_listing_records = []

def safe_get_json(url, params=None, headers=None, timeout=20):
    """
    Request JSON and return parsed JSON (or None on failure).
    Preferably the API returns a list; we return what .json() gives and caller should validate.
    """
    try:
        resp = requests.get(url, params=params, headers=headers, timeout=timeout)
    except requests.RequestException as e:
        print(f"Request error for {url} with params {params}: {e}")
        return None

    if resp.status_code != 200:
        print(f"Non-200 response ({resp.status_code}) for {url} with params {params}: {resp.text[:200]}")
        return None

    try:
        return resp.json()
    except ValueError:
        print(f"Failed to parse JSON for {url} with params {params}")
        return None

# Step 1: Fetch sale listings for a broad area to get addresses
# Example: Query by city (e.g., "San Antonio, TX") or zip code. Adjust as needed.
search_params = {
    "zipCode": "78204",  # You can change this to another city, zip code, or other supported parameter
    "limit": 50  # Fetch up to 50 listings to ensure we get enough unique addresses
}

print(f"\n=== Fetching sale listings for {search_params['zipCode']} ===")
sale_listings = safe_get_json(SALE_LISTING_URL, params=search_params, headers=headers)

# Step 2: Extract unique addresses from sale listings
addresses = []
if sale_listings and isinstance(sale_listings, list):
    for listing in sale_listings:
        address = listing.get("formattedAddress")
        if address and address not in addresses:
            addresses.append(address)
            if len(addresses) >= 10:  # Stop at 10 unique addresses
                break
    print(f" -> Found {len(addresses)} unique addresses")
else:
    print(" -> No sale listings returned")
    addresses = []

# Step 3: Fetch property and sale listing data for each address
for address in addresses[:10]:  # Limit to 10 addresses
    print(f"\n=== Getting data for: {address} ===")
    params = {"address": address}

    # Get property data
    prop_json = safe_get_json(BASE_PROPERTY_URL, params=params, headers=headers)
    if prop_json and isinstance(prop_json, list) and len(prop_json) > 0:
        property_records.append(prop_json[0])
        print(" -> Property data appended")
    else:
        print(" -> No property data returned")

    # Get sale listing data
    sale_json = safe_get_json(SALE_LISTING_URL, params=params, headers=headers)
    if sale_json and isinstance(sale_json, list) and len(sale_json) > 0:
        sale_listing_records.append(sale_json[0])
        print(" -> Sale listing appended")
    else:
        print(" -> No sale listing returned")

    # Small pause to be polite to the API
    time.sleep(0.2)

In [50]:
# Convert to DataFrames (safe: check empty lists)
df_properties = pd.json_normalize(property_records) if property_records else pd.DataFrame()
df_sale_listings = pd.json_normalize(sale_listing_records) if sale_listing_records else pd.DataFrame()


print("\n✅ Data extraction complete.")
print("Properties rows:", len(df_properties))
print("Sale listings rows:", len(df_sale_listings))




✅ Data extraction complete.
Properties rows: 10
Sale listings rows: 10


In [6]:
# Show previews
#df_properties.head()
#df_sale_listings.head()


In [7]:
# To get list of columns in a dataframe

#list(df_sale_listings.columns)
#list(df_properties.columns)


#### TRANSFORNATION

In [51]:
# Select the columns you want first 
df_sale_listings = df_sale_listings[[
    'id','status', 'price', 'listingType', 'listedDate','propertyType',
    'listingAgent.name','listingAgent.phone','listingAgent.email',
    'listingOffice.name', 'listingOffice.phone', 'listingOffice.email', 'listingOffice.website',
    'removedDate','createdDate','lastSeenDate'
]]

df_properties = df_properties[[
    'id','formattedAddress','county','lastSaleDate','owner.names','owner.type',
    'lastSalePrice','ownerOccupied','bedrooms','bathrooms','squareFootage','yearBuilt',
    'city','state','zipCode','latitude','longitude','propertyType','lotSize'
]]




In [52]:
# (Optional) rename columns 
df_sale_listings = df_sale_listings.rename(columns={
    "id": "listing_code",
    "listedDate": "listed_Date",
    "listingType": "listing_Type",
    "propertyType": "listing_property_Type",
    "listingAgent.name": "agent_name",
    "listingAgent.phone": "agent_phone",
    "listingAgent.email": "agent_email",
    "listingOffice.name": "listing_office_name",
    "listingOffice.phone": "listing_office_phone",
    "listingOffice.email": "listing_office_email",
    "listingOffice.website": "listing_office_website",
    "removedDate": "removed_Date",
    "createdDate": "created_Date",
    "lastSeenDate": "last_Seen_Date"

})

df_properties = df_properties.rename(columns={
    "id": "property_code",
    "propertyType": "property_Type",
    "owner.names": "owner_names",
    "owner.type": "owner_type",
    "lastSaleDate": "last_saleDate",
    "lastSalePrice": "last_SalePrice",
    "ownerOccupied": "owner_Occupied",
    "squareFootage": "square_Footage",
    "yearBuilt": "year_Built",
    "zipCode": "zip_Code",
    "lotSize": "lot_Size",
    "formattedAddress": "property_Address"
})

In [53]:
# Convert Date columns 
df_properties['last_saleDate'] = pd.to_datetime(df_properties['last_saleDate'], errors='coerce')

# Convert multiple date columns at once
date_cols = ['listed_Date', 'removed_Date', 'created_Date', 'last_Seen_Date']
df_sale_listings[date_cols] = df_sale_listings[date_cols].apply(pd.to_datetime, errors='coerce')



In [11]:
# GETTING DATAFRAME INFORMATION

#df_properties.info()
#df_sale_listings.info()

In [54]:
#filling missing values

# Fill rules for df_properties
df_properties.fillna({
    'bathrooms': 0.0,
    'bedrooms': 0.0,
    'square_Footage': 0.0,
    'county': 'unknown',
    'property_Type': 'unknown',
    'year_Built': 0.0,  
    'lot_Size': 0.0,
    'owner_Occupied': 0.0,
    'last_SalePrice': 0.0,
    'owner_names': 'unknown',
    'owner_type': 'unknown',
    'Address': 'unknown',
    'city': 'unknown',
    'state': 'unknown',
    'zip_Code': 'unknown',
    'latitude': 0.0,
    'longitude': 0.0,
    'last_saleDate': pd.NaT
}, inplace=True)



In [56]:
# Fill rules for df_sale_listings
df_sale_listings.fillna({
    'status': 'unknown',
    'price': 0.0,
    'listing_Type': 'unknown',
    'listed_Date': pd.NaT,
    'listing_property_Type': 'unknown',
    'agent_name': 'unknown',
    'agent_phone': 'unknown',
    'agent_email': 'unknown',
    'listing_office_name': 'unknown',
    'listing_office_phone': 'unknown',
    'listing_office_email': 'unknown',
    'listing_office_website': 'unknown',
    'removed_Date': pd.NaT,
    'created_Date': pd.NaT,
    'last_Seen_Date': pd.NaT
    
}, inplace=True);

In [57]:
import pandas as pd

# --- 1) Merge safely (use left to keep all properties; change to 'inner' if needed) ---
df_merged = pd.merge(
    df_properties,
    df_sale_listings,
    left_on='property_code',
    right_on='listing_code',
    how='left',
    suffixes=('', '_listing'),
    indicator=True
)

print("Merged shape:", df_merged.shape)

Merged shape: (10, 36)


In [15]:
#df_merged.columns

In [58]:
# drop duplicate column
df_merged.drop(['listing_property_Type'], axis=1, inplace=True)


In [63]:
# # Save to CSVs
df_merged.to_csv("property_data.csv", index=False)

print("\nFiles saved: property_data.csv")


Files saved: property_data.csv


In [2]:
# read csv so  you dont keep calling the api
df_primesquare = pd.read_csv("property_data.csv")

#### CREATE DIMENSION TABLES

In [3]:
# Create the directory if it doesn't exist
os.makedirs("cleaned_data", exist_ok=True)

In [83]:
location_dim_table = df_primesquare[['city', 'state', 'zip_Code','county',
                              'longitude', 'latitude']].copy().drop_duplicates().reset_index(drop=True)
# to add a column that was not in existence use the below code
location_dim_table.index.name = 'location_id'
# USE THIS CODE TO MAKE THE NEW CREATED INDEX TO APPEAR ON THE COLUMN HEADER
location_dim_table = location_dim_table.reset_index()
location_dim_table.to_csv("cleaned_data/location_dim.csv", index=False)
print(f"CSV file created at: {os.path.abspath('cleaned_data')}")
#print(location_dim_table.head())

CSV file created at: c:\Users\back2\Desktop\primesqure_API_Project\src\cleaned_data


In [84]:
property_dim_table = df_primesquare[['property_code', 'property_Address', 'property_Type',
                                'bedrooms', 'bathrooms', 'square_Footage', 'year_Built',
                                'lot_Size']].copy().drop_duplicates().reset_index(drop=True)
property_dim_table.index.name = 'property_id'
property_dim_table = property_dim_table.reset_index()
property_dim_table.to_csv("cleaned_data/property_dim_table.csv", index=False)
#print(property_dim_table.head())

In [85]:
agent_dim_table = df_primesquare[['agent_name', 'agent_phone', 'agent_email']].copy().drop_duplicates().reset_index(drop=True)
agent_dim_table.index.name = 'agent_id'
# USE THIS CODE TO MAKE THE NEW CREATED INDEX TO APPEAR ON THE COLUMN HEADER
agent_dim_table = agent_dim_table.reset_index()
agent_dim_table.to_csv("cleaned_data/agent_data.csv" , index=False)
#print(agent_dim_table.head())

In [68]:
# Convert lists in owner_names to strings
df_primesquare['owner_names'] = df_primesquare['owner_names'].apply(
    lambda x: ', '.join(x) if isinstance(x, list) else x
)

In [86]:
owner_dim_table = df_primesquare[['owner_names', 'owner_type', 'owner_Occupied']].copy().drop_duplicates().reset_index(drop=True)
owner_dim_table.index.name = 'owner_id'
# USE THIS CODE TO MAKE THE NEW CREATED INDEX TO APPEAR ON THE COLUMN HEADER
owner_dim_table = owner_dim_table.reset_index()
owner_dim_table.to_csv("cleaned_data/owner_data.csv", index=False)

In [87]:
office_dim_table = df_primesquare[['listing_office_name', 'listing_office_phone', 'listing_office_email',
                               'listing_office_website']].copy().drop_duplicates().reset_index(drop=True)
office_dim_table.index.name = 'office_id'
office_dim_table = office_dim_table.reset_index()
office_dim_table.to_csv("cleaned_data/office_data.csv", index=False)
#print(office_dim_table.head())

In [88]:
listing_dim_table = df_primesquare[['listing_code','listing_Type',]].copy().drop_duplicates().reset_index(drop=True)
listing_dim_table.index.name = 'listing_id'
listing_dim_table = listing_dim_table.reset_index()
listing_dim_table.to_csv("cleaned_data/listing_data.csv", index=False)
#print(listing_dim_table.head())

In [89]:
# Drop the '_merge' column from df_primesquare to avoid confusion
if '_merge' in df_primesquare.columns:
    df_primesquare = df_primesquare.drop(columns=['_merge'])

# Map foreign keys from dimension tables
fact_dim_table = (df_primesquare.merge(property_dim_table, 
                                      on=['property_code', 'property_Address', 'property_Type', 
                                          'bedrooms', 'bathrooms', 'square_Footage', 'year_Built', 'lot_Size'], 
                                      how='left')
                               .merge(owner_dim_table, 
                                      on=['owner_names', 'owner_type', 'owner_Occupied'], 
                                      how='left')
                               .merge(location_dim_table, 
                                      on=['city', 'state', 'zip_Code', 'county', 'longitude', 'latitude'], 
                                      how='left')
                               .merge(agent_dim_table, 
                                      on=['agent_name', 'agent_phone', 'agent_email'], 
                                      how='left')
                               .merge(office_dim_table, 
                                      on=['listing_office_name', 'listing_office_phone', 
                                          'listing_office_email', 'listing_office_website'], 
                                      how='left')
                               .merge(listing_dim_table, 
                                      on=['listing_code', 'listing_Type'], 
                                      how='left'))

# Select columns from fact_dim_table (not df_primesquare)
# Assuming dimension tables provide 'property_id', 'owner_id', etc.
selected_columns = [
    'property_id',  
    'owner_id',     
    'location_id',  
    'agent_id',     
    'office_id',    
    'listing_id',   
    'status', 
    'price', 
    'listing_Type', 
    'listed_Date', 
    'last_saleDate', 
    'removed_Date', 
    'created_Date', 
    'last_Seen_Date', 
    'property_Type',  
    'last_SalePrice'
]

# Select available columns and handle missing ones
available_columns = [col for col in selected_columns if col in fact_dim_table.columns]
fact_dim_table = fact_dim_table[available_columns].copy()

# Add fact_id if it doesn't exist
if 'fact_id' not in fact_dim_table.columns:
    fact_dim_table['fact_id'] = fact_dim_table.index + 1  # Simple incremental ID

# Reorder columns to include fact_id first
fact_dim_table = fact_dim_table[['fact_id'] + [col for col in available_columns if col != 'fact_id']]

# Drop duplicates and reset index
fact_dim_table = fact_dim_table.drop_duplicates().reset_index(drop=True)

# Set fact_id as index name (if needed)
fact_dim_table.index.name = 'fact_id'
fact_dim_table.to_csv("cleaned_data/fact_data.csv", index=False)

# Print the first few rows to verify
#print(fact_dim_table.head())

In [8]:
# DEVELOPE A FUNCTION TO GET THE DATABASE CONNECTION
# Load environment variables from .env file
load_dotenv()

def get_db_connection():
    connection = psycopg2.connect(
        host=os.getenv('P_host'),
        database=os.getenv('P_database'),
        user=os.getenv('p_user'),
        password=os.getenv('P_password'),
        port=os.getenv('P_port')  # Add the port here
    ) 
    return connection

# connect to database
conn = get_db_connection()

#### Create pstgreSQL tables and schema for database connection

In [10]:
def create_tables():
    conn = get_db_connection()
    cursor = conn.cursor()
    create_table_query = '''
        CREATE SCHEMA IF NOT EXISTS primesquare;

        DROP TABLE IF EXISTS primesquare.property_dim_table CASCADE;
        DROP TABLE IF EXISTS primesquare.owner_dim_table CASCADE;
        DROP TABLE IF EXISTS primesquare.location_dim_table CASCADE;
        DROP TABLE IF EXISTS primesquare.agent_dim_table CASCADE;
        DROP TABLE IF EXISTS primesquare.office_dim_table CASCADE;
        DROP TABLE IF EXISTS primesquare.listing_dim_table CASCADE;
        DROP TABLE IF EXISTS primesquare.fact_dim_table CASCADE;

        CREATE TABLE IF NOT EXISTS primesquare.property_dim_table (
            property_id SERIAL PRIMARY KEY,
            property_code VARCHAR(50) NOT NULL,
            property_address VARCHAR(255) NOT NULL,
            property_type VARCHAR(50),
            bedrooms INTEGER,
            bathrooms INTEGER,
            square_footage INTEGER,
            year_built INTEGER,
            lot_size INTEGER,
            UNIQUE (property_code, property_address, property_type, bedrooms, bathrooms, square_footage, year_built, lot_size)
        );

        CREATE TABLE IF NOT EXISTS primesquare.owner_dim_table (
            owner_id SERIAL PRIMARY KEY,
            owner_names VARCHAR(500) NOT NULL,
            owner_type VARCHAR(100),
            owner_occupied BOOLEAN,
            UNIQUE (owner_names, owner_type, owner_occupied)
        );

        CREATE TABLE IF NOT EXISTS primesquare.location_dim_table (
            location_id SERIAL PRIMARY KEY,
            city VARCHAR(100),
            state VARCHAR(50),
            zip_code VARCHAR(20),
            county VARCHAR(100),
            longitude DOUBLE PRECISION,
            latitude DOUBLE PRECISION,
            UNIQUE (city, state, zip_code, county, longitude, latitude)
        );

        CREATE TABLE IF NOT EXISTS primesquare.agent_dim_table (
            agent_id SERIAL PRIMARY KEY,
            agent_name VARCHAR(255) NOT NULL,
            agent_phone VARCHAR(20),
            agent_email VARCHAR(255),
            UNIQUE (agent_name, agent_phone, agent_email)
        );

        CREATE TABLE IF NOT EXISTS primesquare.office_dim_table (
            office_id SERIAL PRIMARY KEY,
            listing_office_name VARCHAR(255) NOT NULL,
            listing_office_phone VARCHAR(20),
            listing_office_email VARCHAR(255),
            listing_office_website VARCHAR(255),
            UNIQUE (listing_office_name, listing_office_phone, listing_office_email, listing_office_website)
        );

        CREATE TABLE IF NOT EXISTS primesquare.listing_dim_table (
            listing_id SERIAL PRIMARY KEY,
            listing_code VARCHAR(50) NOT NULL,
            listing_type VARCHAR(50),
            UNIQUE (listing_code, listing_type)
        );

        CREATE TABLE IF NOT EXISTS primesquare.fact_dim_table (
            fact_id SERIAL PRIMARY KEY,
            property_id INTEGER NOT NULL,
            owner_id INTEGER,
            location_id INTEGER,
            agent_id INTEGER,
            office_id INTEGER,
            listing_id INTEGER,
            status VARCHAR(50),
            price DECIMAL(15, 2),
            listing_type VARCHAR(50),
            listed_date TIMESTAMP,
            last_saleDate TIMESTAMP,
            removed_date TIMESTAMP,
            created_date TIMESTAMP,
            last_seen_date TIMESTAMP,
            property_Type VARCHAR(100),
            last_sale_price DECIMAL(15, 2),
            FOREIGN KEY (property_id) REFERENCES primesquare.property_dim_table (property_id),
            FOREIGN KEY (owner_id) REFERENCES primesquare.owner_dim_table (owner_id),
            FOREIGN KEY (location_id) REFERENCES primesquare.location_dim_table (location_id),
            FOREIGN KEY (agent_id) REFERENCES primesquare.agent_dim_table (agent_id),
            FOREIGN KEY (office_id) REFERENCES primesquare.office_dim_table (office_id),
            FOREIGN KEY (listing_id) REFERENCES primesquare.listing_dim_table (listing_id)
        );

        -- indexes (schema-qualified)
        CREATE INDEX IF NOT EXISTS idx_fact_primesquare_property_id 
            ON primesquare.fact_dim_table (property_id);

        CREATE INDEX IF NOT EXISTS idx_fact_primesquare_owner_id 
            ON primesquare.fact_dim_table (owner_id);

        CREATE INDEX IF NOT EXISTS idx_fact_primesquare_location_id 
            ON primesquare.fact_dim_table (location_id);

        CREATE INDEX IF NOT EXISTS idx_fact_primesquare_agent_id 
            ON primesquare.fact_dim_table (agent_id);

        CREATE INDEX IF NOT EXISTS idx_fact_primesquare_office_id 
            ON primesquare.fact_dim_table (office_id);

        CREATE INDEX IF NOT EXISTS idx_fact_primesquare_listing_id 
            ON primesquare.fact_dim_table (listing_id);

        CREATE INDEX IF NOT EXISTS idx_fact_primesquare_status 
            ON primesquare.fact_dim_table (status);

        CREATE INDEX IF NOT EXISTS idx_fact_primesquare_listed_date 
            ON primesquare.fact_dim_table (listed_date);
    '''
    cursor.execute(create_table_query)
    conn.commit()
    cursor.close()
    conn.close()


In [11]:
create_tables()

In [12]:
# Define database parameters including the database name
# Load environment variables from .env file
# Load environment variables from .env file

load_dotenv()

# Define database parameters
db_params = {
    'P_user': os.getenv("P_user"),
    'P_password': os.getenv("P_password"),
    'P_host': os.getenv("P_host"),
    'P_port': os.getenv("P_port"),
    'P_database': os.getenv("P_database")
}

# Define the default database URL for PostgreSQL
default_db_url = f"postgresql://{db_params['P_user']}:{db_params['P_password']}@{db_params['P_host']}:{db_params['P_port']}/postgres"

try:
    # Connect to the default database
    conn = psycopg2.connect(default_db_url)
    conn.autocommit = True
    cur = conn.cursor()

    # Check if the target database already exists
    cur.execute(f"SELECT 1 FROM pg_catalog.pg_database WHERE datname = %s", (db_params['P_database'],))
    exists = cur.fetchone()

    if not exists:
        # Create the target database if it doesn't exist
        cur.execute(f"CREATE DATABASE {db_params['P_database']}")
        print(f"P_database '{db_params['database']}' created successfully.")
    else:
        print(f"Database '{db_params['P_database']}' already exists.")

    # Close cursor and connection
    cur.close()
    conn.close()
    
except Exception as e:
    print(f"An error occurred: {e}")


Database 'primesquare_db' already exists.


In [13]:
from dotenv import load_dotenv
import os

load_dotenv()  # Ensure this is executed

# Debugging: Print loaded environment variables
print("P_user':", os.getenv("P_user"))
print("P_password:", os.getenv("P_password"))
print("P_host:", os.getenv("P_host"))
print("P_port:", os.getenv("P_port"))
print("P_database:", os.getenv("P_database"))


P_user': postgres
P_password: mypayment55
P_host: localhost
P_port: 5432
P_database: primesquare_db


#### Insert data into database created tables and schema

In [23]:
import csv


def load_data_from_csv(csv_path):
    conn = get_db_connection()
    cursor = conn.cursor()
    
    with open(csv_path, 'r') as file:
        reader = csv.reader(file)
        next(reader)  # skip header
        
        for row in reader:
            # Convert numeric fields to int or float as needed
            property_id = int(row[0])
            property_code = row[1]
            property_address = row[2]
            property_type = row[3]
            bedrooms = int(float(row[4]))      # "2.0" -> 2
            bathrooms = int(float(row[5]))     # "1.0" -> 1
            square_footage = int(float(row[6])) # "1200.0" -> 1200
            year_built = int(float(row[7]))
            lot_size = float(row[8])           # keep float if needed
            
            cursor.execute(
                '''
                INSERT INTO primesquare.property_dim_table(
                    property_id, property_code, property_address, property_type, 
                    bedrooms, bathrooms, square_footage, year_built, lot_size
                )
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
                ''',
                (property_id, property_code, property_address, property_type,
                 bedrooms, bathrooms, square_footage, year_built, lot_size)
            )
    
    conn.commit()
    cursor.close()
    conn.close()
# Provide the CSV path
csv_file_path = r'C:\Users\back2\Desktop\primesqure_API_Project\src\cleaned_data\property_dim_table.csv'

load_data_from_csv(csv_file_path)

In [20]:
import pandas as pd

def load_owner_dim_table_from_csv(csv_path):
    # Load CSV into DataFrame
    df = pd.read_csv(csv_path)
    
    conn = get_db_connection()
    cursor = conn.cursor()
    
    for _, row in df.iterrows():
        owner_id = int(row['owner_id'])  # convert to int
        owner_names = row['owner_names']
        owner_type = row['owner_type']
        
        # Convert owner_occupied to boolean
        if isinstance(row['owner_Occupied'], str):
            owner_occupied = row['owner_Occupied'].strip().lower() in ['1', 'true', 'yes']
        else:
            owner_occupied = bool(row['owner_Occupied'])
        
        cursor.execute(
            '''
            INSERT INTO primesquare.owner_dim_table(
                owner_id, owner_names, owner_type, owner_occupied
            )
            VALUES (%s, %s, %s, %s)
            ''',
            (owner_id, owner_names, owner_type, owner_occupied)
        )
    
    conn.commit()
    cursor.close()
    conn.close()
    print("Owner data loaded successfully.")

# Provide the CSV path
csv_file_path = r'C:\Users\back2\Desktop\primesqure_API_Project\src\cleaned_data\owner_data.csv'

load_owner_dim_table_from_csv(csv_file_path)


Owner data loaded successfully.


In [29]:
import pandas as pd
import psycopg2
from psycopg2.extras import execute_values

def load_location_dim_table_from_csv(csv_path):
    try:
        # Load CSV into DataFrame
        df = pd.read_csv(csv_path)
        
        # Validate expected columns
        expected_columns = ['location_id', 'city', 'state', 'zip_Code', 'county', 'longitude', 'latitude']
        missing_columns = [col for col in expected_columns if col not in df.columns]
        if missing_columns:
            raise ValueError(f"Missing columns in CSV: {missing_columns}")
        
        # Convert data types
        df['location_id'] = pd.to_numeric(df['location_id'], errors='coerce').fillna(0).astype(int)
        df['zip_Code'] = df['zip_Code'].astype(str)  # Ensure zip_code is string to preserve leading zeros
        df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce').fillna(0.0)
        df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce').fillna(0.0)
        
        # Get database connection
        conn = get_db_connection()
        cursor = conn.cursor()
        
        try:
            # Prepare data for bulk insert
            records = [
                (
                    row['location_id'],
                    row['city'],
                    row['state'],
                    row['zip_Code'],
                    row['county'],
                    row['longitude'],
                    row['latitude']
                )
                for _, row in df.iterrows()
            ]
            
            # Bulk insert
            execute_values(
                cursor,
                '''
                INSERT INTO primesquare.location_dim_table(
                    location_id, city, state, zip_code, county, longitude, latitude
                )
                VALUES %s
                ''',
                records
            )
            
            conn.commit()
            print(f"Successfully loaded {len(records)} rows into location_dim_table.")
        
        except psycopg2.Error as e:
            print(f"Database error: {e}")
            conn.rollback()
            raise
        finally:
            cursor.close()
            conn.close()
    
    except FileNotFoundError:
        print(f"CSV file not found: {csv_path}")
        raise
    except pd.errors.EmptyDataError:
        print(f"CSV file is empty: {csv_path}")
        raise
    except Exception as e:
        print(f"Error processing CSV: {e}")
        raise

# Provide the CSV path
csv_file_path = r'C:\Users\back2\Desktop\primesqure_API_Project\src\cleaned_data\location_dim.csv'
load_location_dim_table_from_csv(csv_file_path)

Successfully loaded 10 rows into location_dim_table.


In [30]:
import pandas as pd

def load_agent_dim_table_from_csv(csv_path):
    # Load CSV into DataFrame
    df = pd.read_csv(csv_path)
    
    conn = get_db_connection()
    cursor = conn.cursor()
    
    for _, row in df.iterrows():
        agent_id = int(row['agent_id'])  # convert to int
        agent_name = row['agent_name']
        agent_phone = row['agent_phone']
        agent_email = row['agent_email']
        
        cursor.execute(
            '''
            INSERT INTO primesquare.agent_dim_table(
                agent_id, agent_name, agent_phone, agent_email
            )
            VALUES (%s, %s, %s, %s)
            ''',
            (agent_id, agent_name, agent_phone, agent_email)
        )
    
    conn.commit()
    cursor.close()
    conn.close()
    print("Agent data loaded successfully.")

# Provide the CSV path
csv_file_path = r'C:\Users\back2\Desktop\primesqure_API_Project\src\cleaned_data\agent_data.csv'

load_agent_dim_table_from_csv(csv_file_path)


Agent data loaded successfully.


In [31]:
import pandas as pd

def load_office_dim_table_from_csv(csv_path):
    # Load CSV into DataFrame
    df = pd.read_csv(csv_path)
    
    conn = get_db_connection()
    cursor = conn.cursor()
    
    for _, row in df.iterrows():
        office_id = int(row['office_id'])  # convert to int
        listing_office_name = row['listing_office_name']
        listing_office_phone = row['listing_office_phone']
        listing_office_email = row['listing_office_email']
        listing_office_website = row['listing_office_website']
        
        cursor.execute(
            '''
            INSERT INTO primesquare.office_dim_table(
                office_id, listing_office_name, listing_office_phone, listing_office_email, listing_office_website
            )
            VALUES (%s, %s, %s, %s, %s)
            ''',
            (office_id, listing_office_name, listing_office_phone, listing_office_email, listing_office_website)
        )
    
    conn.commit()
    cursor.close()
    conn.close()
    print("Office data loaded successfully.")

# Provide the CSV path
csv_file_path = r'C:\Users\back2\Desktop\primesqure_API_Project\src\cleaned_data\office_data.csv'

load_office_dim_table_from_csv(csv_file_path)


Office data loaded successfully.


In [34]:
import pandas as pd

def load_listing_dim_table_from_csv(csv_path):
    # Load CSV into DataFrame
    df = pd.read_csv(csv_path)
    
    conn = get_db_connection()
    cursor = conn.cursor()
    
    for _, row in df.iterrows():
        listing_id = int(row['listing_id'])  # convert to int
        listing_code = row['listing_code']
        listing_Type = row['listing_Type']
        
        cursor.execute(
            '''
            INSERT INTO primesquare.listing_dim_table(
                listing_id, listing_code, listing_Type
            )
            VALUES (%s, %s, %s)
            ''',
            (listing_id, listing_code, listing_Type)
        )
    
    conn.commit()
    cursor.close()
    conn.close()
    print("Listing data loaded successfully.")

# Provide the CSV path
csv_file_path = r'C:\Users\back2\Desktop\primesqure_API_Project\src\cleaned_data\listing_data.csv'

load_listing_dim_table_from_csv(csv_file_path)


Listing data loaded successfully.


In [45]:
import pandas as pd
import psycopg2
from psycopg2.extras import execute_values

def load_fact_dim_table_from_csv(csv_path):
    try:
        # Load CSV into DataFrame
        df = pd.read_csv(csv_path)
        
        # Validate expected columns
        expected_columns = [
            'fact_id', 'property_id', 'owner_id', 'location_id', 'agent_id', 'office_id', 'listing_id',
            'status', 'price', 'listing_Type', 'listed_Date', 'last_saleDate', 'removed_Date',
            'created_Date', 'last_Seen_Date', 'last_SalePrice', 'property_Type'
        ]
        missing_columns = [col for col in expected_columns if col not in df.columns]
        if missing_columns:
            raise ValueError(f"Missing columns in CSV: {missing_columns}")
        
        # Convert data types
        df['fact_id'] = pd.to_numeric(df['fact_id'], errors='coerce').fillna(0).astype(int)
        df['property_id'] = pd.to_numeric(df['property_id'], errors='coerce').fillna(0).astype(int)
        df['owner_id'] = pd.to_numeric(df['owner_id'], errors='coerce').fillna(0).astype(int)
        df['location_id'] = pd.to_numeric(df['location_id'], errors='coerce').fillna(0).astype(int)
        df['agent_id'] = pd.to_numeric(df['agent_id'], errors='coerce').fillna(0).astype(int)
        df['office_id'] = pd.to_numeric(df['office_id'], errors='coerce').fillna(0).astype(int)
        df['listing_id'] = pd.to_numeric(df['listing_id'], errors='coerce').fillna(0).astype(int)
        df['price'] = pd.to_numeric(df['price'], errors='coerce').fillna(pd.NA)
        df['last_SalePrice'] = pd.to_numeric(df['last_SalePrice'], errors='coerce').fillna(pd.NA)
        
        # Date fields: Ensure they are in a format compatible with PostgreSQL (e.g., YYYY-MM-DD)
        date_columns = ['listed_Date', 'last_saleDate', 'removed_Date', 'created_Date', 'last_Seen_Date']
        for col in date_columns:
            df[col] = pd.to_datetime(df[col], errors='coerce').dt.strftime('%Y-%m-%d')
            df[col] = df[col].where(df[col].notnull(), None)
        
        # Get database connection
        conn = get_db_connection()
        cursor = conn.cursor()
        
        try:
            # Prepare data for bulk insert
            records = [
                (
                    row['fact_id'],
                    row['property_id'],
                    row['owner_id'],
                    row['location_id'],
                    row['agent_id'],
                    row['office_id'],
                    row['listing_id'],
                    row['status'],
                    row['price'],
                    row['listing_Type'],
                    row['listed_Date'],
                    row['last_saleDate'],
                    row['removed_Date'],
                    row['created_Date'],
                    row['last_Seen_Date'],
                    row['last_SalePrice'],
                    row['property_Type']
                )
                for _, row in df.iterrows()
            ]
            
            # Bulk insert (database columns use lowercase)
            execute_values(
                cursor,
                '''
                INSERT INTO primesquare.fact_dim_table(
                    fact_id, property_id, owner_id, location_id, agent_id, office_id, listing_id,
                    status, price, listing_type, listed_date, last_saledate, removed_date,
                    created_date, last_seen_date, last_sale_price, property_type
                )
                VALUES %s
                ''',
                records
            )
            
            conn.commit()
            print(f"Successfully loaded {len(records)} rows into fact_dim_table.")
        
        except psycopg2.Error as e:
            print(f"Database error: {e}")
            conn.rollback()
            raise
        finally:
            cursor.close()
            conn.close()
    
    except FileNotFoundError:
        print(f"CSV file not found: {csv_path}")
        raise
    except pd.errors.EmptyDataError:
        print(f"CSV file is empty: {csv_path}")
        raise
    except Exception as e:
        print(f"Error processing CSV: {e}")
        raise

# Provide the CSV path
csv_file_path = r'C:\Users\back2\Desktop\primesqure_API_Project\src\cleaned_data\fact_data.csv'
load_fact_dim_table_from_csv(csv_file_path)

Successfully loaded 10 rows into fact_dim_table.


##### CALL STORED PROCEDURE

In [ ]:
import psycopg2
from datetime import datetime

# Connect to the database
conn = psycopg2.connect(
    host="localhost",
    database="primesquare",
    user="postgres",
    password="your_password"
)
cur = conn.cursor()

# Call the procedure
property_id = 101
new_status = 'Removed'
removed_date = datetime.now()

cur.execute(
    "CALL primesquare.update_property_status(%s, %s, %s);",
    (property_id, new_status, removed_date)
)

conn.commit()
cur.close()
conn.close()
